# Snow-cover attributes and time-series extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the snow-cover time-series from the MODIS dataset.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Observations
* This notebook assumes that the GEE code to export snow-cover mean time-series from the MODIS dataset were run before in the GEE platform and that the output CSV-files are locally available. 
* It is not possible to export the 15,000 catchments at one single CSV, so there might be many files with the time-series stored separetly. 

## Requirements
**Python:**
* Python>=3.6
* Jupyter
* geopandas=0.10.2
* glob
* numpy
* os
* pandas
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**
* data/shapefiles/estreams_catchments.shp
* data/gee/snowcover/EStreams_modis_snow_cover_mean_gee_{}.csv. Snow-cover time-series CSV-file(s) exported from GEE

**Directory:**
* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References

* Hall, D. K., V. V. Salomonson, and G. A. Riggs. 2016. MODIS/Terra Snow Cover Daily L3 Global 500m Grid. Version 6. Boulder, Colorado USA: NASA National Snow and Ice Data Center Distributed Active Archive Center.

## Licenses

* Snow Cover: Open access: You may download and use photographs, imagery, or text from the NSIDC web site, unless limitations for its use are specifically stated. For more information on usage and citing NSIDC datasets, please visit the [NSIDC 'Use and Copyright' page] (https://nsidc.org/about/data-use-and-copyright).(Last access 27 November 2023) 

## Observations
* This notebook assumes that the GEE code to export snow cover time-series from the MODIS dataset (EStreams_landscape_timeseries_Snow_Cover_gee.txt) was run before in the GEE platform and that the output CSV-files are locally available. 

# Import modules

In [2]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tqdm as tqdm
import glob

# Configurations

In [6]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

* #### The users should NOT change anything in the code below here.

In [7]:
# Non-editable variables:
PATH_OUTPUT_TS = "results/timeseries/snowcover"
PATH_OUTPUT_ST = "results/staticattributes"

# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [ ]:
catchment_boundaries = gpd.read_file('data/shapefiles/estreams_catchments.shp')
catchment_boundaries

,id,area_km2,outlet_lat,outlet_lng,name,area_offic,layer,path,area_diff,area_calc,basin_id,geometry
0,HUGR020,"9,600",46.785,21.142,6444410,9011,HUGR020,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,6.536,9595.794,HUGR020,"POLYGON ((21.13208 46.77291, 21.13208 46.77375..."
1,HUGR021,"189,000",46.423,18.896,6442080,189538,HUGR021,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.284,188597.110,HUGR021,"POLYGON ((18.91708 46.41791, 18.91708 46.41625..."
2,HUGR022,"28,500",48.126,22.340,6444304,29057,HUGR022,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-1.917,28507.473,HUGR022,"POLYGON ((22.32875 48.10875, 22.32791 48.10875..."
3,HUGR023,"188,000",46.627,18.869,6442060,189092,HUGR023,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.577,188286.167,HUGR023,"POLYGON ((18.89041 46.62875, 18.88875 46.62708..."
4,HUGR025,"1,210",47.662,19.683,6444240,1222,HUGR025,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.982,1206.441,HUGR025,"POLYGON ((19.68124 47.66875, 19.68291 47.66875..."
5,HUGR026,110,46.891,20.498,6444420,26647,HUGR026,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-99.587,109.639,HUGR026,"POLYGON ((20.49958 46.93125, 20.49958 46.93125..."
6,HUGR027,"4,490",48.497,21.229,6444330,4515,HUGR027,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.554,4494.402,HUGR027,"POLYGON ((21.23458 48.49708, 21.23208 48.49708..."
7,HUGR028,"5,770",46.883,18.141,6442110,5884,HUGR028,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-1.937,5773.506,HUGR028,"POLYGON ((16.56458 46.93291, 16.56541 46.93291..."
8,HUGR029,"185,000",47.495,19.048,6442040,184893,HUGR029,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,0.058,184810.677,HUGR029,"POLYGON ((19.11291 47.48291, 19.11125 47.48458..."
9,HUGR030,"13,000",46.419,16.695,6446100,13033,HUGR030,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.253,13044.665,HUGR030,"POLYGON ((16.63625 46.45625, 16.63541 46.45625..."


## GEE outputs
### Snow-cover

In [9]:
# Check the files in the subdirectory:
filenames = glob.glob("data/gee/snowcover/*.csv")
print("Number of files:", len(filenames))
print("First file:", filenames[0])

Number of files: 1
First file: data/gee/snowcover/EStreams_modis_snow_cover_mean_gee_0_49.csv


In [10]:
# First, we create an empty DataFrame for the data with a datetime index:
snowcover_df = pd.DataFrame(index=pd.date_range(start='2001-01-01', end='2022-12-31', freq='M'))

# Loop for reading and concatenating the data:
for file in tqdm.tqdm(filenames):
    
    # Read the data from the CSV file:
    snowcover_file = pd.read_csv(file)
    snowcover_file.drop(["system:index", ".geo"], axis=1, inplace=True)
    snowcover_file = snowcover_file.T
    
    # Set columns based on the "basin_id" row and drop it
    snowcover_file.columns = snowcover_file.loc["basin_id", :].tolist()
    snowcover_file.drop(["basin_id"], axis=0, inplace=True)
    
    # Convert the index to integers and sort it
    snowcover_file.index = snowcover_file.index.astype(int)
    snowcover_file.sort_index(inplace=True)
    
    # Create a new DataFrame with datetime index and assign values
    snowcover_file_df = pd.DataFrame(columns=snowcover_file.columns)
    snowcover_file_df["dates"] = pd.date_range(start='2001-01-01', end='2022-12-31', freq='M')
    snowcover_file_df.loc[:, snowcover_file.columns] = snowcover_file
    snowcover_file_df.set_index("dates", inplace=True)
    snowcover_file_df.index.name = ""
    
    # Concatenate the DataFrames along the columns (axis=1)
    snowcover_df = pd.concat([snowcover_df, snowcover_file_df], axis=1)
    
snowcover_df

/var/folders/_g/v_sd2vds19n_f5ckymsg1h4r0000gn/T/ipykernel_75970/1304977038.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  snowcover_df = pd.DataFrame(index=pd.date_range(start='2001-01-01', end='2022-12-31', freq='M'))
  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/_g/v_sd2vds19n_f5ckymsg1h4r0000gn/T/ipykernel_75970/1304977038.py:22: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  snowcover_file_df["dates"] = pd.date_range(start='2001-01-01', end='2022-12-31', freq='M')
100%|██████████| 1/1 [00:00<00:00, 54.21it/s]


,HUGR019,HUGR024,HUGR034,HUGR041,HUGR051,HUGR030,HUGR021,HUGR023,HUGR042,HUGR047,...,HUGR029,HUGR025,HUGR037,HUGR032,HUGR022,HUGR039,HUGR036,HUGR050,HUGR048,HUGR027
2001-01-31,16.501745,6.132638,31.5716,34.905246,38.115947,38.938548,36.49363,36.55011,1.865271,3.302784,...,37.197997,1.440321,13.300634,37.425884,25.872987,2.948067,3.133586,11.551791,22.911232,23.932382
2001-02-28,14.851006,10.764342,19.759875,20.858095,22.052805,20.499629,19.585523,19.610039,7.264862,7.962198,...,19.884561,1.552536,3.091635,19.99339,20.780148,9.329599,10.98645,10.481494,20.284269,17.83369
2001-03-31,3.691307,5.192324,15.410848,16.944892,18.556173,17.418341,13.192397,13.213184,0.06683,0.603388,...,13.453082,0.677347,1.867396,13.537813,5.531266,4.154976,4.140561,1.340782,4.894459,6.856402
2001-04-30,0.675242,0.074507,11.144205,12.377205,13.574608,9.840066,6.88601,6.896999,0.015591,0.051145,...,7.023452,0.046306,0.654172,7.068244,1.377724,0.042332,0.04541,0.113257,1.211342,0.178523
2001-05-31,0.074573,0.037923,4.809941,5.341017,5.858368,2.41305,2.998075,3.002867,0.025474,0.053884,...,3.058047,0.0,0.303793,3.077611,0.136546,0.010362,0.016164,0.037036,0.120609,0.029301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.077912,0.214951,0.265301,0.293233,0.321473,0.142995,0.120994,0.121187,0.02164,0.022814,...,0.123394,0.073632,0.077155,0.124139,0.103673,0.101122,0.164374,0.029572,0.091093,0.06122
2022-09-30,0.468483,0.41951,0.960255,1.050068,1.148512,0.882601,1.032894,1.034544,0.580296,0.232764,...,1.052484,0.060838,0.204314,1.058939,0.654979,0.187008,0.299071,0.466369,0.588142,0.399727
2022-10-31,0.173398,0.162251,0.811585,0.887704,0.971944,0.373129,0.632471,0.633241,0.030263,0.01205,...,0.644356,0.059177,0.437668,0.647183,0.214077,0.056316,0.025144,0.145671,0.206739,0.223216
2022-11-30,2.043442,1.344197,9.243607,10.221935,11.090784,8.839096,5.872911,5.881139,0.216112,0.557992,...,5.964118,0.951897,1.276005,5.995996,3.021715,0.689197,0.951985,1.431932,2.744894,1.951848


In [11]:
# Here we add the columns of the catchemnts that were not processed
# Adding new columns with NaN values only if they don't exist
for col in catchment_boundaries.basin_id.tolist():
    if col not in snowcover_df.columns:
        snowcover_df[col] = np.nan
snowcover_df

,HUGR019,HUGR024,HUGR034,HUGR041,HUGR051,HUGR030,HUGR021,HUGR023,HUGR042,HUGR047,...,HUGR029,HUGR025,HUGR037,HUGR032,HUGR022,HUGR039,HUGR036,HUGR050,HUGR048,HUGR027
2001-01-31,16.501745,6.132638,31.5716,34.905246,38.115947,38.938548,36.49363,36.55011,1.865271,3.302784,...,37.197997,1.440321,13.300634,37.425884,25.872987,2.948067,3.133586,11.551791,22.911232,23.932382
2001-02-28,14.851006,10.764342,19.759875,20.858095,22.052805,20.499629,19.585523,19.610039,7.264862,7.962198,...,19.884561,1.552536,3.091635,19.99339,20.780148,9.329599,10.98645,10.481494,20.284269,17.83369
2001-03-31,3.691307,5.192324,15.410848,16.944892,18.556173,17.418341,13.192397,13.213184,0.06683,0.603388,...,13.453082,0.677347,1.867396,13.537813,5.531266,4.154976,4.140561,1.340782,4.894459,6.856402
2001-04-30,0.675242,0.074507,11.144205,12.377205,13.574608,9.840066,6.88601,6.896999,0.015591,0.051145,...,7.023452,0.046306,0.654172,7.068244,1.377724,0.042332,0.04541,0.113257,1.211342,0.178523
2001-05-31,0.074573,0.037923,4.809941,5.341017,5.858368,2.41305,2.998075,3.002867,0.025474,0.053884,...,3.058047,0.0,0.303793,3.077611,0.136546,0.010362,0.016164,0.037036,0.120609,0.029301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.077912,0.214951,0.265301,0.293233,0.321473,0.142995,0.120994,0.121187,0.02164,0.022814,...,0.123394,0.073632,0.077155,0.124139,0.103673,0.101122,0.164374,0.029572,0.091093,0.06122
2022-09-30,0.468483,0.41951,0.960255,1.050068,1.148512,0.882601,1.032894,1.034544,0.580296,0.232764,...,1.052484,0.060838,0.204314,1.058939,0.654979,0.187008,0.299071,0.466369,0.588142,0.399727
2022-10-31,0.173398,0.162251,0.811585,0.887704,0.971944,0.373129,0.632471,0.633241,0.030263,0.01205,...,0.644356,0.059177,0.437668,0.647183,0.214077,0.056316,0.025144,0.145671,0.206739,0.223216
2022-11-30,2.043442,1.344197,9.243607,10.221935,11.090784,8.839096,5.872911,5.881139,0.216112,0.557992,...,5.964118,0.951897,1.276005,5.995996,3.021715,0.689197,0.951985,1.431932,2.744894,1.951848


In [12]:
# Here we sort the columns:
snowcover_df = snowcover_df.sort_index(axis=1)

snowcover_df

,HUGR019,HUGR020,HUGR021,HUGR022,HUGR023,HUGR024,HUGR025,HUGR026,HUGR027,HUGR028,...,HUGR042,HUGR043,HUGR044,HUGR045,HUGR046,HUGR047,HUGR048,HUGR049,HUGR050,HUGR051
2001-01-31,16.501745,2.375188,36.49363,25.872987,36.55011,6.132638,1.440321,1.139817,23.932382,8.804724,...,1.865271,1.395941,15.688291,12.332657,3.990662,3.302784,22.911232,0.854437,11.551791,38.115947
2001-02-28,14.851006,7.421254,19.585523,20.780148,19.610039,10.764342,1.552536,0.0,17.83369,7.026,...,7.264862,15.846653,14.123678,11.244701,7.211757,7.962198,20.284269,1.818834,10.481494,22.052805
2001-03-31,3.691307,0.302586,13.192397,5.531266,13.213184,5.192324,0.677347,0.011253,6.856402,2.294656,...,0.06683,0.088467,3.545108,1.523411,0.484097,0.603388,4.894459,0.003285,1.340782,18.556173
2001-04-30,0.675242,0.031009,6.88601,1.377724,6.896999,0.074507,0.046306,0.139814,0.178523,2.369286,...,0.015591,0.00661,0.66237,0.575061,0.130451,0.051145,1.211342,0.018298,0.113257,13.574608
2001-05-31,0.074573,0.035729,2.998075,0.136546,3.002867,0.037923,0.0,0.0,0.029301,2.119218,...,0.025474,0.018991,0.075037,0.138992,0.049551,0.053884,0.120609,0.000109,0.037036,5.858368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.077912,0.020196,0.120994,0.103673,0.121187,0.214951,0.073632,0.0,0.06122,0.062531,...,0.02164,0.008404,0.07661,0.129152,0.037596,0.022814,0.091093,0.0,0.029572,0.321473
2022-09-30,0.468483,0.37367,1.032894,0.654979,1.034544,0.41951,0.060838,0.000926,0.399727,1.49348,...,0.580296,0.130119,0.45248,0.417039,0.337111,0.232764,0.588142,0.078648,0.466369,1.148512
2022-10-31,0.173398,0.040675,0.632471,0.214077,0.633241,0.162251,0.059177,0.0,0.223216,1.434521,...,0.030263,0.096391,0.173338,0.068421,0.040935,0.01205,0.206739,0.041527,0.145671,0.971944
2022-11-30,2.043442,0.364273,5.872911,3.021715,5.881139,1.344197,0.951897,1.427293,1.951848,3.877217,...,0.216112,0.300754,2.002167,1.657981,0.598414,0.557992,2.744894,0.152824,1.431932,11.090784


In [13]:
# Resample to yearly mean
snowcover_yr = snowcover_df.resample('Y').mean()

snowcover_yr

/var/folders/_g/v_sd2vds19n_f5ckymsg1h4r0000gn/T/ipykernel_75970/3619369021.py:2: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  snowcover_yr = snowcover_df.resample('Y').mean()


,HUGR019,HUGR020,HUGR021,HUGR022,HUGR023,HUGR024,HUGR025,HUGR026,HUGR027,HUGR028,...,HUGR042,HUGR043,HUGR044,HUGR045,HUGR046,HUGR047,HUGR048,HUGR049,HUGR050,HUGR051
2001-12-31,7.17138,4.896968,12.082765,9.736347,12.094216,5.504417,4.998141,6.080167,8.518071,6.11787,...,4.824641,6.522647,7.030585,7.218827,5.383143,4.984316,9.165966,5.466031,4.811177,12.924104
2002-12-31,7.324148,2.928304,8.564252,9.704672,8.567351,5.321186,2.364306,5.387015,6.653825,5.780027,...,3.016574,2.884278,7.080632,6.298434,3.31122,3.20307,8.896525,1.387388,6.984853,10.766953
2003-12-31,13.809576,10.399498,12.153639,15.939514,12.154434,10.15889,8.607365,12.482326,12.420623,12.123146,...,9.701039,12.603777,13.685899,13.576693,11.365836,10.72781,15.477593,12.502624,13.574302,15.185914
2004-12-31,11.608395,6.719321,13.334235,14.517039,13.35207,8.954749,5.217386,2.192825,11.201241,7.220785,...,6.971272,5.962893,11.188199,9.574535,6.685418,7.475735,13.691325,3.956868,11.480175,14.965165
2005-12-31,12.04932,8.516115,15.934886,15.520105,15.94896,9.1774,3.835396,9.183384,11.435499,8.788464,...,8.527133,7.411299,11.735214,11.289478,8.885264,8.348809,14.632004,7.278554,11.059566,16.182556
2006-12-31,11.451033,5.931014,15.875795,13.091043,15.89865,8.500224,6.922882,4.804476,12.377497,7.851529,...,6.038363,6.231139,11.339716,8.20978,5.92155,6.402619,12.644878,3.379139,11.834508,18.242661
2007-12-31,5.258104,1.69316,8.084106,7.51222,8.095132,2.255572,1.07112,0.147223,6.807702,3.051767,...,1.610771,1.980692,5.017439,5.133743,2.205019,2.092317,6.780028,0.940669,4.654928,11.273853
2008-12-31,5.357766,1.794496,7.760028,6.740662,7.767505,4.371765,2.236953,3.288169,6.997317,3.684741,...,1.78925,0.530614,5.232534,4.924525,2.363221,1.84103,6.01269,0.99187,5.101408,11.855059
2009-12-31,9.203701,7.707761,12.755775,10.488133,12.771884,6.53268,6.78996,5.736163,10.637917,5.072023,...,7.279526,7.597854,9.142691,10.543234,8.192559,8.230184,10.224372,7.047176,9.035348,15.594121
2010-12-31,10.157915,9.556084,16.76519,10.440236,16.771148,9.716237,9.080945,14.044232,12.292596,10.457575,...,8.643631,9.74591,10.103411,10.969819,10.272732,10.119605,10.239604,11.849983,10.244402,17.521024


In [14]:
# Calculate the mean for each month across all years (monht of the year)
snowcover_moy = snowcover_df.groupby(snowcover_df.index.month).mean()

# Rename the index to the three-letter month abbreviations
snowcover_moy.index = pd.to_datetime(snowcover_moy.index, format='%m').strftime('%b')

snowcover_moy

,HUGR019,HUGR020,HUGR021,HUGR022,HUGR023,HUGR024,HUGR025,HUGR026,HUGR027,HUGR028,...,HUGR042,HUGR043,HUGR044,HUGR045,HUGR046,HUGR047,HUGR048,HUGR049,HUGR050,HUGR051
Jan,33.476857,20.768741,34.384505,39.303477,34.417933,26.16165,17.081754,20.035496,35.53797,16.688199,...,19.752867,21.410469,32.783116,30.274221,22.176271,22.777834,37.264451,18.712226,33.786415,36.111063
Feb,27.2252,16.574574,30.083076,31.728376,30.10912,21.567287,13.812452,19.214287,30.809572,16.081587,...,16.193347,15.623263,26.577484,22.530244,17.603482,17.122966,30.142284,15.475598,28.305403,35.097667
Mar,11.064629,4.208772,14.725461,15.993179,14.745016,5.344069,2.584726,3.880856,10.420081,5.771538,...,3.808392,4.597541,10.661622,10.290988,5.392658,4.757437,14.672105,4.782324,9.474648,22.562773
Apr,1.538934,0.267329,5.667688,3.053621,5.67669,0.330792,0.0373,0.021713,0.529845,2.453166,...,0.151455,0.052457,1.467508,1.713437,0.501897,0.438209,2.689072,0.024187,0.41805,10.977033
May,0.161279,0.07226,2.539158,0.266674,2.543216,0.145876,0.056848,0.002456,0.104903,1.96802,...,0.053836,0.027,0.15904,0.225532,0.093798,0.097116,0.237652,0.015499,0.078959,4.278829
Jun,0.065386,0.058835,0.842377,0.086194,0.843699,0.05453,0.030158,0.02217,0.11085,1.056618,...,0.055078,0.033041,0.065912,0.105301,0.060859,0.069736,0.078036,0.017131,0.045394,1.34831
Jul,0.072077,0.051403,0.286465,0.09535,0.286916,0.070833,0.019358,0.008781,0.092155,0.725084,...,0.046154,0.02793,0.070703,0.155386,0.065079,0.066816,0.087149,0.025211,0.055614,0.465027
Aug,0.064796,0.024873,0.223527,0.075274,0.223762,0.091649,0.029206,0.009676,0.079568,0.795997,...,0.025421,0.01222,0.06372,0.079577,0.033934,0.024692,0.069217,0.015198,0.058095,0.389727
Sep,0.110075,0.048324,0.621192,0.129522,0.622177,0.094398,0.027278,0.008134,0.156494,1.4078,...,0.050325,0.040921,0.108744,0.082662,0.05147,0.054324,0.123208,0.039315,0.105687,0.992679
Oct,0.55917,0.086354,2.09378,0.872489,2.097039,0.281258,0.119514,0.019268,0.516971,2.219827,...,0.100858,0.060577,0.539701,0.338368,0.129167,0.078615,0.777812,0.054293,0.410951,3.854769


# Final aggregation (static attributes)

In [15]:
# Snow cover:
snowcover_moy_T = snowcover_moy.T
snowcover_moy_T.columns = pd.to_datetime(snowcover_moy_T.columns, format='%b').strftime('%m')
snowcover_moy_T.columns = "sno_cov_" + snowcover_moy_T.columns
snowcover_moy_T["sno_cov_mean"] = snowcover_moy_T.mean(axis = 1)
snowcover_moy_T

,sno_cov_01,sno_cov_02,sno_cov_03,sno_cov_04,sno_cov_05,sno_cov_06,sno_cov_07,sno_cov_08,sno_cov_09,sno_cov_10,sno_cov_11,sno_cov_12,sno_cov_mean
HUGR019,33.476857,27.2252,11.064629,1.538934,0.161279,0.065386,0.072077,0.064796,0.110075,0.55917,3.483508,19.10777,8.077473
HUGR020,20.768741,16.574574,4.208772,0.267329,0.07226,0.058835,0.051403,0.024873,0.048324,0.086354,0.98741,11.120309,4.522432
HUGR021,34.384505,30.083076,14.725461,5.667688,2.539158,0.842377,0.286465,0.223527,0.621192,2.09378,8.93021,24.819104,10.434712
HUGR022,39.303477,31.728376,15.993179,3.053621,0.266674,0.086194,0.09535,0.075274,0.129522,0.872489,4.911139,23.255708,9.980917
HUGR023,34.417933,30.10912,14.745016,5.67669,2.543216,0.843699,0.286916,0.223762,0.622177,2.097039,8.943736,24.841532,10.445903
HUGR024,26.16165,21.567287,5.344069,0.330792,0.145876,0.05453,0.070833,0.091649,0.094398,0.281258,2.812627,16.906603,6.155131
HUGR025,17.081754,13.812452,2.584726,0.0373,0.056848,0.030158,0.019358,0.029206,0.027278,0.119514,1.419755,10.953071,3.847618
HUGR026,20.035496,19.214287,3.880856,0.021713,0.002456,0.02217,0.008781,0.009676,0.008134,0.019268,0.529084,8.951824,4.391979
HUGR027,35.53797,30.809572,10.420081,0.529845,0.104903,0.11085,0.092155,0.079568,0.156494,0.516971,4.742137,24.98072,9.006772
HUGR028,16.688199,16.081587,5.771538,2.453166,1.96802,1.056618,0.725084,0.795997,1.4078,2.219827,3.887453,10.721467,5.31473


In [16]:
# Assign the "basin_id" to the gauges names:
snowcover_moy_T.index.name = "basin_id"

In [17]:
# Assign the "date" as the index name of the df:
snowcover_yr.index.name = "date"
snowcover_df.index.name = "date"

In [28]:
# Round the data to 3 decimals
snowcover_df = snowcover_df.astype(float).round(3)
snowcover_yr = snowcover_yr.astype(float).round(3)
snowcover_moy_T = snowcover_moy_T.astype(float).round(3)

# Data export

In [31]:
# Export the final datasets:
# Time-series:
snowcover_df.to_csv(PATH_OUTPUT_TS+"/estreams_snowcover_monhtlye.csv")
snowcover_yr.to_csv(PATH_OUTPUT_TS+"/estreams_snowcover_yearlye.csv")

# Static attributes:
snowcover_moy_T.to_csv(PATH_OUTPUT_ST+"/estreams_snowcover_attributese.csv")

# End